In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value, current_dow30_symbols
import numpy as np
import pandas as pd

In [13]:
symbol="AMD"
timeInterval = "1m"
longTimeInterval = "1wk"
timeRange = "3d"

In [14]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

https://query1.finance.yahoo.com/v8/finance/chart/AMD?symbol=AMD&region=US&lang=en-US&includePrePost=false&interval=1m&useYfid=true&range=3d&corsDomain=finance.yahoo.com&.tsrc=finance


{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614978002,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 78.52,
 'chartPreviousClose': 84.13,
 'previousClose': 77.75,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614934800,
   'end': 1614954600,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614954600,
   'end': 1614978000,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614978000,
   'end': 1614992400,
   'gmtoffset': -18000}},
 'tradingPeriods': [[{'timezone': 'EST',
    'start': 1614781800,
    'end': 1614805200,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614868200,
    'end': 1614891600,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614954600,
    'end': 1614978000,
    'gmtoffset': -18000}]],


In [15]:
longMetadataAndPrice = metadata_price_action(symbol, longTimeInterval, timeRange)
longMetadata = metadataAndPrice["metadata"]
longMetadata

https://query1.finance.yahoo.com/v8/finance/chart/AMD?symbol=AMD&region=US&lang=en-US&includePrePost=false&interval=1wk&useYfid=true&range=3d&corsDomain=finance.yahoo.com&.tsrc=finance


{'currency': 'USD',
 'symbol': 'AMD',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 322151400,
 'regularMarketTime': 1614978002,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 78.52,
 'chartPreviousClose': 84.13,
 'previousClose': 77.75,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614934800,
   'end': 1614954600,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614954600,
   'end': 1614978000,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614978000,
   'end': 1614992400,
   'gmtoffset': -18000}},
 'tradingPeriods': [[{'timezone': 'EST',
    'start': 1614781800,
    'end': 1614805200,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614868200,
    'end': 1614891600,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614954600,
    'end': 1614978000,
    'gmtoffset': -18000}]],


In [16]:
from typing import Dict, List, Optional, Sequence, Union
from numbers import Number

def _data_period(index) -> Union[pd.Timedelta, Number]:
    """Return data index period as pd.Timedelta"""
    values = pd.Series(index[-100:])
    return values.diff().dropna().median()


In [18]:
price = metadataAndPrice["price"]
# price.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
price = price.dropna()
price["Close"]=price["close"]
price["High"]=price["high"]
price["Low"]=price["low"]
price["Open"]=price["open"]
price["Volume"]=price["volume"]
del price['close']
del price['high']
del price['low']
del price['open']
del price['volume']

price['timestamp']=price["timestamp"].apply(lambda x: pd.Timestamp(x, unit='s'))
price['timestamp']=pd.DatetimeIndex(price["timestamp"])

price = price.set_index('timestamp')
# price = pd.read_csv("index_problem.csv", index_col=0, parse_dates=True)
price

<ipython-input-18-2db47b9b4ff1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price["Close"]=price["close"]
<ipython-input-18-2db47b9b4ff1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price["High"]=price["high"]
<ipython-input-18-2db47b9b4ff1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,Close,High,Low,Open,Volume
timestamp,,,,,
2021-03-03 14:30:00,83.739998,83.739998,83.720001,83.730003,788612.0
2021-03-03 14:31:00,84.019997,84.059998,83.720001,83.760002,149055.0
2021-03-03 14:32:00,84.054802,84.139900,83.800003,84.000000,208125.0
2021-03-03 14:33:00,83.879997,84.120003,83.824997,84.030403,228635.0
2021-03-03 14:34:00,83.970001,84.089897,83.889999,83.900002,118392.0
...,...,...,...,...,...
2021-03-05 20:55:00,78.367302,78.610001,78.367302,78.525002,218000.0
2021-03-05 20:56:00,78.532303,78.541702,78.360001,78.368500,252572.0
2021-03-05 20:57:00,78.544998,78.580101,78.489998,78.540001,229184.0


In [19]:
longPrice = longMetadataAndPrice["price"]
# longPrice.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
longPrice = longPrice.dropna()
longPrice["Close"]=longPrice["close"]
longPrice["High"]=longPrice["high"]
longPrice["Low"]=longPrice["low"]
longPrice["Open"]=longPrice["open"]
longPrice["Volume"]=longPrice["volume"]
del longPrice['close']
del longPrice['high']
del longPrice['low']
del longPrice['open']
del longPrice['volume']

longPrice['timestamp']=longPrice["timestamp"].apply(lambda x: pd.Timestamp(x, unit='s').date())
longPrice['timestamp']=pd.DatetimeIndex(longPrice["timestamp"])

longPrice = longPrice.set_index('timestamp')
# price = pd.read_csv("index_problem.csv", index_col=0, parse_dates=True)

longPrice

,Close,High,Low,Open,Volume
timestamp,,,,,
2021-03-01,78.519997,84.379997,74.199997,84.279999,162208500
2021-03-05,78.519997,79.480003,74.199997,79.000000,58548499


In [22]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
from statistics import mean
# print(GOOG, GOOG.index.dtype, GOOG.dtypes)
def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 8) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)
    
class SmaCross(Strategy):
    def init(self):
        tempprice = self.data.Close
        self.ma1 = self.I(myEMA, tempprice, 8)
        self.ma2 = self.I(myEMA, tempprice, 13)
        self.ma3 = self.I(myEMA, tempprice, 21)

        self.macd = self.I(myMacd, tempprice)
        self.macdSignal = self.I(myMacdSignal, tempprice)
        self.macdHist = self.I(myMacdHist, tempprice)

    def next(self):
        if(not self.position and self.macdHist[-1] > 0):
            self.buy()
        elif(self.macdHist[-1] < 0):
            self.position.close()
#         if crossover(self.ma1, self.ma2):
#             self.buy()
#         elif crossover(self.ma2, self.ma1):
#             self.sell()

# print(price, price.index.dtype, price.dtypes)
print("Buy and hold: ", round(100 + ((price.Close[-1]-price.Close[0])/price.Close[0])*100), "%")
bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

Buy and hold:  94 %


In [61]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '77.75',
 'Open': '79.00',
 'Bid': '78.88 x 1000',
 'Ask': '78.90 x 900',
 "Day's Range": '74.24 - 79.48',
 '52 Week Range': '36.75 - 99.23',
 'Volume': '58,103,511',
 'Avg. Volume': '42,843,283',
 'Market Cap': '95.113B',
 'Beta (5Y Monthly)': '2.18',
 'PE Ratio (TTM)': '38.04',
 'EPS (TTM)': '2.06',
 'Earnings Date': 'Apr 26, 2021 - Apr 30, 2021',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Ex-Dividend Date': 'Apr 27, 1995'}

In [62]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['101.91B',
  '111.16B',
  '98.55B',
  '61.76B',
  '53.26B',
  '53.66B'],
 'Enterprise Value 3': ['100.15B',
  '109.97B',
  '97.67B',
  '61.08B',
  '52.48B',
  '53.56B'],
 'Trailing P/E ': ['40.84', '123.93', '157.67', '122.35', '151.60', '241.37'],
 'Forward P/E 1': ['44.25', '51.02', '49.26', '52.63', '39.84', '43.48'],
 'PEG Ratio (5 yr expected) 1': ['1.57',
  '1.13',
  '1.42',
  '1.55',
  '1.66',
  '1.55'],
 'Price/Sales (ttm)': ['10.40', '12.68', '12.67', '8.37', '7.57', '8.38'],
 'Price/Book (mrq)': ['17.46', '28.72', '29.82', '20.34', '18.84', '24.66'],
 'Enterprise Value/Revenue 3': ['10.26',
  '33.90',
  '34.87',
  '31.61',
  '29.39',
  '25.18'],
 'Enterprise Value/EBITDA 6': ['59.76',
  '167.63',
  '193.41',
  '238.59',
  '202.63',
  '180.94'],
 'Beta (5Y Monthly) ': '2.18',
 '52-Week Change 3': '81.47%',
 'S&P500 52-Week Change 3': '39.88%',
 '52 Week High 3': '99.23',
 '52 Week Low 3': '36.75',
 '50-Day Moving Average 3': '88.06',
 '200-Day Movi